In [1]:
ALGORITHM_NAME = 'multi-modelrf-sagar'

In [11]:
%%sh

# The name of our algorithm
algorithm_name=multi-model-random-forest-new

cd container

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
#region=$(aws configure get region)
region=${region:-ap-south-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.
# aws sagemaker list-models

#docker image ls

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}#${fullname}

docker push ${fullname}

Login Succeeded

Step 1/13 : FROM ubuntu:16.04
 ---> 005d2078bdfa
Step 2/13 : LABEL com.amazonaws.sagemaker.capabilities.multi-models=true
 ---> Using cache
 ---> f3c0e6cdd684
Step 3/13 : LABEL com.amazonaws.sagemaker.capabilities.accept-bind-to-port=true
 ---> Using cache
 ---> 078858966e26
Step 4/13 : RUN apt-get update &&     apt-get -y install --no-install-recommends     build-essential     ca-certificates     openjdk-8-jdk-headless     python3-dev     curl     vim     && rm -rf /var/lib/apt/lists/*     && curl -O https://bootstrap.pypa.io/get-pip.py     && python3 get-pip.py
 ---> Using cache
 ---> 60771f033d5b
Step 5/13 : RUN update-alternatives --install /usr/bin/python python /usr/bin/python3 1
 ---> Using cache
 ---> aef26bbbd902
Step 6/13 : RUN update-alternatives --install /usr/local/bin/pip pip /usr/local/bin/pip3 1
 ---> Using cache
 ---> decdd3ebea72
Step 7/13 : RUN pip3 --no-cache-dir install mxnet                                 multi-model-server                       

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Error parsing reference: "580246529711.dkr.ecr.ap-south-1.amazonaws.com/multi-model-random-forest-new#580246529711.dkr.ecr.ap-south-1.amazonaws.com/multi-model-random-forest-new" is not a valid repository/tag: invalid reference format
An image does not exist locally with the tag: 580246529711.dkr.ecr.ap-south-1.amazonaws.com/multi-model-random-forest-new


In [4]:
import numpy as np
import pandas as pd
import json
import datetime
import time
from time import gmtime, strftime
import matplotlib.pyplot as plt

#Train multiple house value prediction models

import sagemaker
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer
import boto3
import boto3
import pandas as pd
from sagemaker import get_execution_role
from sklearn.model_selection import train_test_split

In [11]:

def gen_hous():
    bucket_name = 'sagemaker-data-icr'
    s3 = boto3.resource('s3')
    data_key = 'first phenomes and genomes.csv'
    data_location = 's3://{}/{}'.format(bucket_name, data_key)
    df = pd.read_csv(data_location)
    #data = df.drop(['Unnamed: 0'],axis=1)
    df1 = df.fillna(df.mean())
    COLUMNS = list(df1.columns)
    
    i = str(input("enter a feature name "))
    #for i in col:
    return i,df1



sm_client = boto3.client(service_name='sagemaker')
runtime_sm_client = boto3.client(service_name='sagemaker-runtime')

s3 = boto3.resource('s3')
s3_client = boto3.client('s3')

sagemaker_session = sagemaker.Session()
role = get_execution_role()

ACCOUNT_ID  = boto3.client('sts').get_caller_identity()['Account']
REGION      = boto3.Session().region_name
BUCKET      = sagemaker_session.default_bucket()
SCRIPT_FILENAME     = 'train_prediction.py'
USER_CODE_ARTIFACTS = 'gp_code2.tar.gz'



DATA_PREFIX            = 'gp_rf2'
END_MODEL_NAME     = 'MXNET-endpoints'
MULTI_MODEL_ARTIFACTS  = 'multi_model_artifacts'

TRAIN_INSTANCE_TYPE    = 'ml.m4.xlarge'
ENDPOINT_INSTANCE_TYPE = 'ml.m5.xlarge'
#Split a given dataset into train, validation, and test





In [12]:
from sagemaker.sklearn.estimator import SKLearn

def launch_training_job(i):
    # clear out old versions of the data
   # _s3_bucket = s3.Bucket(BUCKET)
   # _full_input_prefix = '{}/model_prep/{}'.format(DATA_PREFIX, location)
   # _s3_bucket.objects.filter(Prefix=_full_input_prefix + '/').delete()

    # upload the entire set of data for all three channels
    #_local_folder = 'data/{}'.format(location)
    inputs = sagemaker_session.upload_data('data')
    print('Training data uploaded: {}'.format(inputs))
    
    _job = 'gp-{}'.format(i.replace('_', '-'))
    _full_output_prefix = '{}/model_artifacts/{}'.format(DATA_PREFIX, 
                                                        i)
    _s3_output_path = 's3://{}/{}'.format(BUCKET, _full_output_prefix)
    return _s3_output_path,_job

    
    


In [13]:
import shutil
import os
LOCATIONS  = ['100SDW_EIAR_2008_RF','100SDW_EIAR_2009_RF','100SDW_EU_2008_RF','100SDW_EU_2009_RF']
PARALLEL_TRAINING_JOBS = 1
training_jobs = []
from sagemaker.sklearn.estimator import SKLearn
for loc in LOCATIONS[:PARALLEL_TRAINING_JOBS]:
    i ,df1=gen_hous() 
    #df1= train_validate_test_split(df1)
    #save_data_locally(loc,df1)
   # _s3_output_path,_job = launch_training_job(loc)
    # clear out old versions of the data
    
    _s3_output_path,_job = launch_training_job(i)
    _estimator = SKLearn(
         entry_point='train_prediction.py', role=role,
         train_instance_count=1, train_instance_type=TRAIN_INSTANCE_TYPE,
         framework_version='0.20.0',
         output_path=_s3_output_path,
         base_job_name=_job)
    
    DISTRIBUTION_MODE = 'FullyReplicated'
    
    train_input = sagemaker_session.upload_data("data")
    _remote_inputs = {'train': train_input}

    _estimator.fit(_remote_inputs, wait=False)
    training_jobs.append( _estimator.latest_training_job.name)
print('{} training jobs launched: {}'.format(len(training_jobs), training_jobs))

KeyboardInterrupt: 

In [62]:
def wait_for_training_job_to_complete(job_name):
    print('Waiting for job {} to complete...'.format(job_name))
    _resp   = sm_client.describe_training_job(TrainingJobName=job_name)
    _status = _resp['TrainingJobStatus']
    while _status=='InProgress':
        time.sleep(60)
        _resp   = sm_client.describe_training_job(TrainingJobName=job_name)
        _status = _resp['TrainingJobStatus']
        if _status == 'InProgress':
            print('{} job status: {}'.format(job_name, _status))
    print('DONE. Status for {} is {}\n'.format(job_name, _status))




In [63]:
import re
def parse_model_artifacts(model_data_url):
    # extract the s3 key from the full url to the model artifacts
    _s3_key = model_data_url.split('s3://{}/'.format(BUCKET))[1]
    # get the part of the key that identifies the model within the model artifacts folder
    _model_name_plus = _s3_key[_s3_key.find('model_artifacts') + len('model_artifacts') + 1:]
    # finally, get the unique model name (e.g., "NewYork_NY")
    _model_name = re.findall('^(.*?)/', _model_name_plus)[0]
    return _s3_key, _model_name

In [64]:
# make a copy of the model artifacts from the original output of the training job to the place in
# s3 where the multi model endpoint will dynamically load individual models
def deploy_artifacts_to_gp(job_name):
    _resp = sm_client.describe_training_job(TrainingJobName=job_name)
    _source_s3_key, _model_name = parse_model_artifacts(_resp['ModelArtifacts']['S3ModelArtifacts'])
    _copy_source = {'Bucket': BUCKET, 'Key': _source_s3_key}
    _key = '{}/{}/{}.tar.gz'.format(DATA_PREFIX, MULTI_MODEL_ARTIFACTS, _model_name)
    
    print('Copying {} model\n   from: {}\n     to: {}...'.format(_model_name, _source_s3_key, _key))
    s3_client.copy_object(Bucket=BUCKET, CopySource=_copy_source, Key=_key)
    return _key

In [65]:

# copy every model except the first one
for job in training_jobs:
    wait_for_training_job_to_complete(job)
    deploy_artifacts_to_gp(job)

Waiting for job gp-APB-PAT-2001-RF2-2020-06-02-06-03-48-391 to complete...
gp-APB-PAT-2001-RF2-2020-06-02-06-03-48-391 job status: InProgress
gp-APB-PAT-2001-RF2-2020-06-02-06-03-48-391 job status: InProgress
gp-APB-PAT-2001-RF2-2020-06-02-06-03-48-391 job status: InProgress
DONE. Status for gp-APB-PAT-2001-RF2-2020-06-02-06-03-48-391 is Completed

Copying APB_PAT_2001_RF2 model
   from: gp_rf2/model_artifacts/APB_PAT_2001_RF2/gp-APB-PAT-2001-RF2-2020-06-02-06-03-48-391/output/model.tar.gz
     to: gp_rf2/multi_model_artifacts/APB_PAT_2001_RF2.tar.gz...


In [23]:

# When using multi-model endpoints with the Scikit Learn container, we need to provide an entry point for
# inference that will at least load the saved model. This function uploads a model artifact containing such a
# script. This tar.gz file will be fed to the SageMaker multi-model creation and pointed to by the 
# SAGEMAKER_SUBMIT_DIRECTORY environment variable.

def upload_inference_code(script_file_name, prefix):
    _tmp_folder = 'inference-code'
    if not os.path.exists(_tmp_folder):
        os.makedirs(_tmp_folder)
    !tar -czvf $_tmp_folder/$USER_CODE_ARTIFACTS $script_file_name > /dev/null
    _loc = sagemaker_session.upload_data(_tmp_folder, 
                                         key_prefix='{}/{}'.format(prefix, _tmp_folder))
    return _loc + '/' + USER_CODE_ARTIFACTS

In [15]:
def create_multi_model_entity(multi_model_name, role):
    # establish the place in S3 from which the endpoint will pull individual models
    _model_url  = 's3://{}/{}/{}/'.format(BUCKET, DATA_PREFIX, MULTI_MODEL_ARTIFACTS)
    _container = {
        'Image':        MULTI_MODEL_SKLEARN_IMAGE,
        'ModelDataUrl': _model_url,
        'Mode':         'MultiModel',
        'Environment': {
            'SAGEMAKER_PROGRAM' : SCRIPT_FILENAME,
            'SAGEMAKER_SUBMIT_DIRECTORY' : upload_inference_code(SCRIPT_FILENAME, DATA_PREFIX)
        }
    }
    create_model_response = sm_client.create_model(
        ModelName = multi_model_name,
        ExecutionRoleArn = role,
        Containers = [_container])
    
    return _model_url

In [16]:
import os
multi_model_name = '{}-{}'.format(HOUSING_MODEL_NAME, strftime('%Y-%m-%d-%H-%M-%S', gmtime()))
model_url = create_multi_model_entity(multi_model_name, role)
print('Multi model name: {}'.format(multi_model_name))

print('Here are the models that the endpoint has at its disposal:')
!aws s3 ls --human-readable --summarize $model_url

Multi model name: MXNET-endpoints-2020-06-08-05-07-42
Here are the models that the endpoint has at its disposal:
2020-06-02 06:02:52   33.7 KiB 100SDW_PAT_2001_RF.tar.gz
2020-06-02 06:07:55   32.0 KiB APB_PAT_2001_RF2.tar.gz

Total Objects: 2
   Total Size: 65.7 KiB


In [17]:
endpoint_config_name = multi_model_name
print('Endpoint config name: ' + endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType': 'ml.m5.xlarge',
        'InitialInstanceCount': 1,
        'InitialVariantWeight': 1,
        'ModelName'   : multi_model_name,
        'VariantName' : 'AllTraffic'}])

endpoint_name = multi_model_name
print('Endpoint name: ' + endpoint_name)

Endpoint config name: MXNET-endpoints-2020-06-08-05-07-42
Endpoint name: MXNET-endpoints-2020-06-08-05-07-42


In [20]:
endpoint_config_name = multi_model_name
print('Endpoint config name: ' + endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType': 'ml.m5.xlarge',
        'InitialInstanceCount': 1,
        'InitialVariantWeight': 1,
        'ModelName'   : multi_model_name,
        'VariantName' : 'AllTraffic'}])

endpoint_name = multi_model_name
print('Endpoint name: ' + endpoint_name)
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print('Endpoint Arn: ' + create_endpoint_response['EndpointArn'])

print('Waiting for {} endpoint to be in service...'.format(endpoint_name))
waiter = sm_client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)


Endpoint config name: MXNET-endpoints-2020-06-08-05-07-42
Endpoint name: MXNET-endpoints-2020-06-08-05-07-42
Endpoint Arn: arn:aws:sagemaker:ap-south-1:580246529711:endpoint/mxnet-endpoints-2020-06-08-05-07-42
Waiting for MXNET-endpoints-2020-06-08-05-07-42 endpoint to be in service...


In [ ]:
import boto3
smrt = boto3.client('sagemaker-runtime')

#endpoint_name = 'multimodel-endpoints-2020-06-05-07-55-36'
def predict_one_house_value(features, model_name):
    print('Using model {} to predict price of this house: {}'.format(model_name,
                                                                     features))

    _float_features = [float(i) for i in features]
    _body = ','.join(map(str, _float_features)) + '\n'
    
    _start_time = time.time()

    _response = smrt.invoke_endpoint(
                        EndpointName=endpoint_name,
                        ContentType='text/csv',
                        TargetModel=model_name,
                        Body=_body)
    _predicted_value = json.loads(_response['Body'].read())[0]

    _duration = time.time() - _start_time
    
    print('${:,.2f}, took {:,d} ms\n'.format(_predicted_value, int(_duration * 1000)))

In [ ]:
import numpy as np
import pandas as pd
import json
import datetime
import time
from time import gmtime, strftime
import matplotlib.pyplot as plt
# iterate through invocations with random inputs against a random model showing results and latency
import numpy as np
for i in range(10):
    full_model_name = '{}.tar.gz'.format('100SDW_PAT_2001_RF')
    out_arr = np.random.randint(low = 0, high = 1099, size = 4799)
    predict_one_house_value(out_arr, full_model_name)

In [19]:
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)


{'ResponseMetadata': {'RequestId': '407e4f19-5074-4512-a246-975dd182e214',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '407e4f19-5074-4512-a246-975dd182e214',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Mon, 08 Jun 2020 05:08:35 GMT'},
  'RetryAttempts': 0}}